<h1>importer la bibiliotéques</h1>

In [1]:
import pandas as pd

<h1>importer le dataset</h1>

In [2]:
data=pd.read_csv("2_dataset_activities_all.csv",  delimiter=';')
data.shape

(7287, 45)

In [3]:
data.isnull().sum().sum()

83405

In [4]:
data.dtypes

Molecule ChEMBL ID             object
Molecule Name                  object
Molecule Max Phase              int64
Molecular Weight              float64
#RO5 Violations                object
AlogP                          object
Compound Key                   object
Smiles                         object
Standard Type                  object
Standard Relation              object
Standard Value                float64
Standard Units                 object
pChEMBL Value                 float64
Data Validity Comment          object
Comment                        object
Uo Units                       object
Ligand Efficiency BEI         float64
Ligand Efficiency LE           object
Ligand Efficiency LLE          object
Ligand Efficiency SEI          object
Potential Duplicate              bool
Assay ChEMBL ID                object
Assay Description              object
Assay Type                     object
BAO Format ID                  object
BAO Label                      object
Assay Organi

<h1>Normalisation</h1>

In [5]:
data=data.rename(columns = {'Molecule ChEMBL ID':'Molecule_ChEMBL_ID','Standard Type':'Standard_Type','Standard Value':'Standard_Value'})

<h1>Choise de bioactivités IC50</h1>

In [6]:
ic50_data=data.query('Standard_Type == "IC50" ')
ic50_data.shape

(1272, 45)

<h1>Drop unused colomns </h1>

In [8]:
ic50_drop_column=ic50_data[['Molecule_ChEMBL_ID','Standard_Value','Smiles']]
ic50_drop_column.head()

,Molecule_ChEMBL_ID,Standard_Value,Smiles
6,CHEMBL3667025,650.0,CC(C)(C)OC(=O)CCC(C(=O)/C=C/c1ccc(O)c(OC(F)(F)...
12,CHEMBL3672702,16.0,CC[C@@H](C)Oc1ccc2c(c1)C1(N=C(C)C(N)=N1)[C@]1(...
13,CHEMBL3672732,4.5,CO[C@H]1CC[C@]2(CCc3ccc(-c4cc(Cl)cc(C#N)c4)cc3...
18,CHEMBL3901022,5400.0,COc1cc(OCc2ccccc2)ccc1/C=C/C(=O)CC(=O)/C=C/c1c...
22,CHEMBL3892054,2200.0,COc1cc(/C=C/C(=O)CC(=O)/C=C/c2ccccc2)ccc1O


<h1>Gérer les valeurs manquantes </h1>

In [ ]:
ic50_data['Standard_Value'].isnull().sum()

130

In [10]:
ic50_drop_column=ic50_drop_column.sort_values(by="Standard_Value" )
index_with_nan = ic50_drop_column.index[ic50_drop_column.iloc[:,1].isnull()]

In [11]:
ic50_data_notnull=ic50_drop_column.drop(index_with_nan,0)
ic50_data_notnull.shape

C:\Users\NEONTECH\AppData\Local\Temp/ipykernel_12608/3926152671.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  ic50_data_notnull=ic50_drop_column.drop(index_with_nan,0)


(1142, 3)

In [12]:
ic50_data_notnull['Standard_Value'].isnull().sum()

0

<h1>Gérer les données du bruit</h1>

In [13]:
negatif_values=ic50_data_notnull.query('Standard_Value <= 0 ')
negatif_values.shape

(2, 3)

In [14]:
ic50_notnull_positive=ic50_data_notnull[ic50_data_notnull.Standard_Value>0]
ic50_notnull_positive.shape

(1140, 3)

<h1>supprimer les lignes dupliquées </h1>

In [15]:
ic50_notnull_positive=ic50_notnull_positive.drop_duplicates(subset=['Molecule_ChEMBL_ID'],keep='last')
ic50_notnull_positive.shape

(960, 3)

In [16]:
ic50_notnull_positive.head()

,Molecule_ChEMBL_ID,Standard_Value,Smiles
3561,CHEMBL240055,0.30,O=C(OC1([C@H]2CCC[C@@H](c3cc(F)cc(F)c3)N2S(=O)...
2479,CHEMBL1091513,0.65,O=S(=O)(NC1CCC(c2cc(F)ccc2F)(S(=O)(=O)c2ccc(Cl...
1208,CHEMBL4110988,0.70,CO[C@H]1CC[C@]2(CC1)Cc1ccc(C#CC(C)C)cc1[C@]21N...
6663,CHEMBL3898248,0.70,CO[C@H]1CC[C@]2(CC1)Cc1ccc(OCCC(F)(F)F)cc1[C@@...
4853,CHEMBL3939963,0.80,CO[C@H]1CC[C@]2(CC1)Cc1ccc(OCCC3CC3)cc1C21N=C(...


<h1>Classification des molecules selon la valeur de ic50</h1>

<h2>Ajouter la colone molecule_class</h2>

In [17]:
ic50_notnull_positive["molecule_class"] = " "

<h2>Active class</h2>

In [18]:
##active=ic50_notnull_positive[ic50_data_notnull.Standard_Value<=1000]
active_class=ic50_notnull_positive.query('Standard_Value  <= 1000 ')
active_class.shape

(370, 4)

In [19]:
active_class.head()

,Molecule_ChEMBL_ID,Standard_Value,Smiles,molecule_class
3561,CHEMBL240055,0.30,O=C(OC1([C@H]2CCC[C@@H](c3cc(F)cc(F)c3)N2S(=O)...,
2479,CHEMBL1091513,0.65,O=S(=O)(NC1CCC(c2cc(F)ccc2F)(S(=O)(=O)c2ccc(Cl...,
1208,CHEMBL4110988,0.70,CO[C@H]1CC[C@]2(CC1)Cc1ccc(C#CC(C)C)cc1[C@]21N...,
6663,CHEMBL3898248,0.70,CO[C@H]1CC[C@]2(CC1)Cc1ccc(OCCC(F)(F)F)cc1[C@@...,
4853,CHEMBL3939963,0.80,CO[C@H]1CC[C@]2(CC1)Cc1ccc(OCCC3CC3)cc1C21N=C(...,


In [20]:
active_class.loc[:,'molecule_class'] ='active'

C:\Users\NEONTECH\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


<h2>Inactive class</h2>

In [21]:
inactive_class=ic50_notnull_positive.query('Standard_Value >= 10000 ')
inactive_class.shape

(368, 4)

In [22]:
inactive_class.loc[:,'molecule_class'] ='inactive'

<h1>Concatination</h1>

In [23]:
dfc_2class=pd.concat([active_class,inactive_class])

In [24]:
dfc_2class.shape

(738, 4)

In [25]:
dfc_2class.head()

,Molecule_ChEMBL_ID,Standard_Value,Smiles,molecule_class
3561,CHEMBL240055,0.30,O=C(OC1([C@H]2CCC[C@@H](c3cc(F)cc(F)c3)N2S(=O)...,active
2479,CHEMBL1091513,0.65,O=S(=O)(NC1CCC(c2cc(F)ccc2F)(S(=O)(=O)c2ccc(Cl...,active
1208,CHEMBL4110988,0.70,CO[C@H]1CC[C@]2(CC1)Cc1ccc(C#CC(C)C)cc1[C@]21N...,active
6663,CHEMBL3898248,0.70,CO[C@H]1CC[C@]2(CC1)Cc1ccc(OCCC(F)(F)F)cc1[C@@...,active
4853,CHEMBL3939963,0.80,CO[C@H]1CC[C@]2(CC1)Cc1ccc(OCCC3CC3)cc1C21N=C(...,active


<h1>Results</h1>

In [26]:
dfc_2class.molecule_class.value_counts()

active      370
inactive    368
Name: molecule_class, dtype: int64

<h1>Exporting data</h1>

In [27]:
#dfc_2class.to_csv("2_dataset_activities_cleaned.csv" ,index=False)